In [115]:
import pandas as pd
import numpy as np
import datetime
import pandas_market_calendars as mcal
from pandas.tseries.offsets import BDay
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
US_BUSINESS_DAY = CustomBusinessDay(calendar=USFederalHolidayCalendar())

Data Loading and Cleaning

In [2]:
InflationRate = pd.read_csv('./data/1yrExpectedInflationRate.csv')
InflationRate.head(5)

In [18]:
CPI = pd.read_csv('./data/cpi.csv')
CPI.head()

,DATE,CPIAUCSL
0,1947-01-01,21.48
1,1947-02-01,21.62
2,1947-03-01,22.00
3,1947-04-01,22.00
4,1947-05-01,21.95


In [17]:
Inventory = pd.read_csv('./data/Inventory.csv',skiprows=4)
Inventory.head()

,Week of,Weekly U.S. Ending Stocks of Crude Oil and Petroleum Products Thousand Barrels
0,03/4/2022,1724594
1,02/25/2022,1735213
2,02/18/2022,1741514
3,02/11/2022,1745787
4,02/4/2022,1758364


In [20]:
USDIndex = pd.read_csv('./data/usd_index.csv')
USDIndex.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1971-01-04,120.529999,120.529999,120.529999,120.529999,120.529999,0.0
1,1971-01-05,120.519997,120.519997,120.519997,120.519997,120.519997,0.0
2,1971-01-06,120.489998,120.489998,120.489998,120.489998,120.489998,0.0
3,1971-01-07,120.550003,120.550003,120.550003,120.550003,120.550003,0.0
4,1971-01-08,120.529999,120.529999,120.529999,120.529999,120.529999,0.0


In [21]:
VIX = pd.read_csv('./data/VIX.csv')
VIX.head()

,DATE,VIXCLS
0,1990-01-02,17.24
1,1990-01-03,18.19
2,1990-01-04,19.22
3,1990-01-05,20.11
4,1990-01-08,20.26


In [22]:
wti = pd.read_excel('./data/PET_PRI_FUT_S1_D.xls', sheet_name=1,skiprows=2)
wti.head()

,Date,"Cushing, OK Crude Oil Future Contract 1 (Dollars per Barrel)","Cushing, OK Crude Oil Future Contract 2 (Dollars per Barrel)","Cushing, OK Crude Oil Future Contract 3 (Dollars per Barrel)","Cushing, OK Crude Oil Future Contract 4 (Dollars per Barrel)"
0,1983-03-30,NaN,NaN,29.35,NaN
1,1983-03-31,NaN,NaN,29.24,NaN
2,1983-04-04,29.44,NaN,29.10,NaN
3,1983-04-05,29.71,NaN,29.35,NaN
4,1983-04-06,29.92,NaN,29.50,NaN


In [180]:
napm = pd.read_csv('data/FRED-NAPM.csv')
napm.head()

,DATE,VALUE
0,2016-05-01,51.3
1,2016-04-01,50.8
2,2016-03-01,51.8
3,2016-02-01,49.5
4,2016-01-01,48.2


In [179]:
napm

,DATE,VALUE
0,2016-05-01,51.3
1,2016-04-01,50.8
2,2016-03-01,51.8
3,2016-02-01,49.5
4,2016-01-01,48.2
...,...,...
816,1948-05-01,49.5
817,1948-04-01,45.4
818,1948-03-01,43.3
819,1948-02-01,50.2
